## 라이브러리 불러오기

In [1]:
import math
from datetime import datetime
import openpyxl
import pandas as pd
from PIL import Image
import numpy as np

### Code 참고 링크

In [2]:
# https://blog.naver.com/PostView.nhn?blogId=rising_n_falling&logNo=222147135486    태양의 고도각, 방위각을 얻는 과정에 대한 전반적인 알고리즘

## Constant

### Unit change

In [3]:
# 각도 단위 변경
d2r = math.pi/180
r2d = 180/math.pi

# 에너지 단위 변경
MJ2J = 1000000    # 10^6 MJ to(2) J

# 시간 단위 변경
one_hour_to_sec = 3600  # 한 시간을 초로
one_hour_to_min = 60 # [min]
one_day_to_hour = 24 # [h]
one_min_to_sec = 60

### Calculation

In [4]:
# ε = long-wave emittance of the surface
# σ = Stefan-Boltzmann constant
# Fgnd = view factor of wall surface to ground surface temperature
# Fsky = view factor of wall surface to sky temperature
# Fair = view factor of wall surface to air temperature
# Tsurf = outside surface temperature
# Tgnd = ground surface temperature
# Tsky = sky temperature
# Tair = air temperature

In [5]:
emissivity_surf = 1.0
boltz_constant = 5.67*10**(-8) 

### 벽체의 법선각

In [6]:
# 벽체 법선각  정남향 0 서쪽 -, 동쪽 +
surf_azimuth = 0  # -- 정남향

### 위도, 실제 경도, 지역경도 입력하기

In [7]:
# 서울 위도, 경도
local_latitude = 37.478
local_longitude = 127.148    # --- 실제 경도
standard_longitude = 135     # --- 일본과 같은 시간을 쓰기 위한 표준 경도 

## Dictionary

In [8]:
# h = D + E*V + F*V^2 
Roughness_dict = {"Very Rough"   : {"D": 11.58, "E" : 5.894, "F" :    0.0}, # Stucoo 
                  "Rough"        : {"D": 12.49, "E" : 4.065, "F" :  0.028}, # Brick
                  "Medium Rough" : {"D": 10.79, "E" : 4.192, "F" :    0.0}, # Concrete
                  "Medium Smooth": {"D":  8.23, "E" : 4.000, "F" : -0.057}, # Clear pine
                  "Smooth"       : {"D": 10.22, "E" : 3.100, "F" :    0.0}, # Smooth Plaster
                  "Very Smooth"  : {"D":  8.23, "E" : 3.300, "F" : -0.036}, # Glass 
                  }

## 함수

### 선형보간 함수

In [9]:
# 선형보간 함수
def interpolate_list(lst,n):
    result = []
    for i in range(len(lst)-1):  # 리스트 길이보다 하나 작은 숫자로 포문을 돌림 -> 마지막에는 포문으로 선형보간 없이 자기 자신만 추가해줘야함 -> 하지만 23:55 까지만 데이터를 얻을 것이기 때문에 추가 따로 안하고 마지막 값은 안들어가는 걸로 했음
        result.append(round(lst[i],2))   # 각 인자들을 빈 리스트인 result에 하나씩 넣음 
        x1, x2 = lst[i], lst[i+1] # 각 x1, x2를 리스트에 있는 i번째 값, i+1번째 값으로 선언 
        y = n + 1                               
        slope = (x2-x1)/y    # x + slope*n 에 쓰일 것
        for i in range(1,n+1):     # x1 데이터 넣고 선형보간한 것 채워넣기
            x2 = x1 + slope*(i) # +1 을 한 이유는 range가 0부터 넣기 때문
            result.append(round(x2,2))
    return result

## 데이터 얻기

### 엑셀 파일경로

### Excel vs CSV
CSV 파일과 Excel 파일의 가장 큰 차이점은 데이터의 형식입니다. CSV 파일은 Comma Separated Values의 약자로서, 데이터를 콤마(,)로 구분하여 저장하는 파일 형식입니다. 반면에, Excel 파일은 Microsoft Excel 프로그램에서 생성되는 전자 스프레드시트 파일 형식입니다.

CSV 파일과 Excel 파일의 차이점은 다음과 같습니다. 첫째, CSV 파일은 데이터를 조작하지 않고 그대로 저장합니다. 반면에 Excel에서는 각 행의 각 열에 시작 태그와 끝 태그가 있어야 합니다 [1]. 둘째, CSV 파일에서는 열 헤더를 한 번만 작성하면 됩니다. 반면에 Excel 파일에서는 각 열의 헤더를 별도로 작성해야 합니다. 셋째, CSV 파일에서 데이터를 가져오는 동안 Excel에서 더 많은 메모리를 사용합니다. 그렇기 때문에 CSV 파일 가져오기 속도가 훨씬 빠릅니다 [1].

따라서, 데이터를 간단하게 저장하고 다룰 때는 CSV 파일을 사용하는 것이 유리하며, 복잡하고 대용량의 데이터를 다룰 때는 Excel 파일이 유용합니다

csv 주의사항, 한국말 못읽음, degree 같은 특수문자 못읽음

### 엑셀에 데이터 저장하기

### New (step sec divide)

In [10]:
#                            년도  시작 달  시작날짜    끝날짜   초 간격    넣을 기상 데이터 파일 위치  저장할 기상 데이터 파일 위치
def save_solar_data_to_excel(year, month, start_day, end_day, step_sec, weather_data_file_location, save_data_file_location ):  # 태양의 고도각, 방위각을 저장하는 함수

    # 시작 시간, 분, 초 모두 0 값으로 세팅 
    local_sec = 0
    local_min = 0
    local_hour = 0

    # 선형보간으로 사이에 추가할 인수 개수
    interpolate_num = one_hour_to_sec/step_sec - 1 

    # ------------------------------------------------------------------------- Data read 
    df_weather_data = pd.read_csv(weather_data_file_location, encoding='cp949')
    df_weather_data['일사(W/m2)'] = df_weather_data['일사(MJ/m2)']*MJ2J/one_hour_to_sec  # Insolation data 단위변경 

    # ------------------------------------------------------------------------- Data to list
    air_temp_vector = df_weather_data['기온(°C)'].to_list()
    ground_surf_temp_vector = df_weather_data['지면온도(°C)'].to_list()
    insolation_vector = df_weather_data['일사(W/m2)'].to_list()
    wind_veolcity_vector = df_weather_data['풍속(m/s)'].to_list()


    time_vector = []                 # 시간에 대한 정보를 저장할 벡터
    solar_altitude_vector = []       # 태양고도각을 저장할 벡터
    solar_azimute_vector = []        # 태양방위각을 저장할 벡터
    angle_between_vector = []        # 벽체 법선각과 태양 방위각의 차이를 저장할 벡터
    convection_heat_transfer_coefficient_vector = [] # wind data를 이용해 대류열전달 계수를 저장할 벡터
    
    current_day = start_day         # 계산에 쓰일 현재 날짜 계산 변수 

    while current_day != end_day+1:  # 현재계산 날짜가 계산 끝날짜 바로 다음날로 넘어가는 순간 정지  

        time_vector.append(f"{year}-{month}-{current_day} {local_hour}:{local_min}:{local_sec}") # 현재 시간을 나타내는 값을 time_vector에 추가해준다

        # -------------------------------------------------------------------------------------------------------------- 균시차 
        day_of_year = datetime(year, month, current_day).timetuple().tm_yday
        B = (day_of_year - 1) * 360/365
        EOT = 229.2 * (0.000075
                    + 0.001868 * math.cos(d2r * B)
                    - 0.032077 * math.sin(d2r * B)
                    - 0.014615 * math.cos(d2r * 2 * B)
                    - 0.04089 * math.sin(d2r * 2 * B))

        # -------------------------------------------------------------------------------------------------------------- 시간각
        local_hour_decimal = local_hour + local_min/60
        delta_longitude = local_longitude - standard_longitude
        solar_time_decimal = (local_hour_decimal * 60 + 4 * delta_longitude + EOT) / 60
        solar_time_hour = int(solar_time_decimal)
        solar_time_min = (solar_time_decimal * 60) % 60
        hour_angle = (local_hour_decimal * 60 + 4 * delta_longitude + EOT) / 60 * 15 - 180

        # -------------------------------------------------------------------------------------------------------------- 태양적위
        solar_declination = 23.45 * math.sin(d2r * 360 / 365 * (284 + day_of_year))

        # -------------------------------------------------------------------------------------------------------------- 태양고도
        term_1 = math.cos(d2r * local_latitude) * math.cos(d2r * solar_declination) * math.cos(d2r * hour_angle) \
                + math.sin(d2r * local_latitude) * math.sin(d2r * solar_declination)
        solar_altitude = r2d * math.asin(term_1)
        solar_altitude_vector.append(solar_altitude) # vector에 값 넣기

        # -------------------------------------------------------------------------------------------------------------- 태양방위각
        term_2 = (math.sin(d2r * solar_altitude) * math.sin(d2r * local_latitude) - math.sin(d2r * solar_declination))\
                / (math.cos(d2r * solar_altitude) * math.cos(d2r * local_latitude))
        solar_azimuth = r2d * math.acos(term_2)
        solar_azimute_vector.append(solar_azimuth)

        # -------------------------------------------------------------------------------------------------------------- 법선 사이각
        angle_between = solar_azimuth - surf_azimuth  # 벽체 법선과 태양 방위각 사이의 각도 
        angle_between_vector.append(angle_between)
        
        # -------------------------------------------------------------------------------------------------------------- 시간 업데이트
        local_sec += step_sec    # step sec 만큼 시간 업데이트 해주기

        if local_sec >= one_min_to_sec:     # 초와 분의 관계 60초 넘어가면 분에 더해주고 나머지 초를 자기 자신으로 업데이트
            local_sec = local_sec-one_min_to_sec
            local_min += 1

        if local_min >= one_hour_to_min:    # 분과 시간 관계 60분 넘어가면 시간에 더해주고 나머지 분을 자기 자신으로 업데이트 
            local_min = local_min-one_hour_to_min
            local_hour += 1

        if local_hour >= one_day_to_hour:   # 시간과 날짜의 관계 24시간 넘어가면 다음날로 업데이트 해주고 남은 시간을 자기자신으로 업데이트 
            local_hour = local_hour - one_day_to_hour
            current_day += 1

    # ----------------------------------------------------------------------------------------------------------------------- 벡터값 선형보간 
    air_temp_interpolated_vector = interpolate_list(air_temp_vector,int(interpolate_num))   # ---- 선형보간으로 분단위 메꿔주기
    ground_surf_temp_interpolated_vector = interpolate_list(ground_surf_temp_vector,int(interpolate_num))
    insolation_interpolated_vector = interpolate_list(insolation_vector,int(interpolate_num))
    wind_veolcity_interpolated_vector = interpolate_list(wind_veolcity_vector,int(interpolate_num))
    
    #----------------------------------------------------------------------------------------------------------------------- 대류열전달 계수 저장
    convection_heat_transfer_coefficient_vector = [Roughness_dict["Medium Rough"]["D"]                                       # h = D + E*V + F*V^2
                                                 + Roughness_dict["Medium Rough"]["E"]*wind_veolcity_interpolated_vector[i]
                                                 + Roughness_dict["Medium Rough"]["F"]*(wind_veolcity_interpolated_vector[i])**2
                                                 for i in range(len(wind_veolcity_interpolated_vector))]
    # ----------------------------------------------------------------------------------------------------------------------- Insolation 벽체에 입사하는 값으로 바꿔주기
    insolation_to_wall_interpolated_vector = [insolation_interpolated_vector[i]
                                              *(math.cos(d2r*solar_altitude_vector[i])
                                               /math.sin(d2r*solar_altitude_vector[i])
                                               *math.cos(d2r*angle_between_vector[i]))
                                              for i in range(len(time_vector))]
    # ----------------------------------------------------------------------------------------------------------------------- Insolation 벽체와 태양의 방위각이 90이상일 때 0값으로 변경해주기
    insolation_to_wall_interpolated_vector = [0 if angle_between_vector[i] >= 90            
                                              else insolation_to_wall_interpolated_vector[i] 
                                              for i in range(len(time_vector))]

    df_solar_data = pd.DataFrame({'Time' : time_vector,
                                  'Solar altitude': solar_altitude_vector,
                                  'Solar azimute' : solar_azimute_vector,
                                  'Between angle' : angle_between_vector,
                                  'Temperature' : air_temp_interpolated_vector,    # -- 기상데이터들은 한 시간단위로 얻어진 것이기 때문에 선형보간을 이용하여 사이를 선형보간으로 메꾸어 추가
                                  'Ground temperature' : ground_surf_temp_interpolated_vector,
                                  'Insolation'         : insolation_interpolated_vector,
                                  'Insolation to wall(W/m2)' : insolation_to_wall_interpolated_vector,
                                  'Wind velocity(m/s)' : wind_veolcity_interpolated_vector, 
                                  'h_c' : convection_heat_transfer_coefficient_vector
                                  })
    df_solar_data.to_csv(save_data_file_location, index= False) # 인덱스 설정 및 최종저장

In [11]:
time_data_dict = {  # 2023년 6월 22일 부터 24일까지 5분 간격으로 태양 고도 방위각 데이터를 얻는 딕셔너리
'year' : 2022,
'month' : 6,
'start_day' : 22,
'end_day' : 24,
'step_sec': 5,
'weather_data_file_location': '../data/Data.csv',
'save_data_file_location': 'New.csv',
}

save_solar_data_to_excel(**time_data_dict) # 함수에 딕셔너리를 넣을 땐 앞에 ** 를 붙여줌

FileNotFoundError: [Errno 2] No such file or directory: 'Data2.csv'